In [12]:
import json
import boto3
import pickle
import pyarrow

import numpy as np
import pandas as pd

from datetime import datetime
from dateutil.relativedelta import relativedelta

from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

from sklearn.feature_extraction import DictVectorizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor

# import xgboost as xgb
from xgboost import XGBRegressor

import mlflow
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType

from hyperopt import STATUS_OK, Trials, fmin, hp, tpe, space_eval
from hyperopt.pyll import scope

from prefect import task, flow
from prefect.task_runners import SequentialTaskRunner

MLFLOW_TRACKING_URI = 'sqlite:///../mlops-project.db'
mlflow.set_tracking_uri("http://127.0.0.1:5000/")

def read_file(key, bucket='kkr-mlops-zoomcamp'):

    session = boto3.session.Session()
    s3 = session.client(
        service_name='s3',
        endpoint_url='https://storage.yandexcloud.net',
        region_name='ru-central1',
        # aws_access_key_id = "id",
        # aws_secret_access_key = "key")
    )
    obj = s3.get_object(Bucket=bucket, Key=key)

    data = pd.read_csv(obj['Body'], sep=",")

    return data

@task
def load_data(current_date = "2015-5-17", periods = 1):
    
    dt_current = datetime.strptime(current_date, "%Y-%m-%d")
    
    if periods == 1:
        date_file = dt_current + relativedelta(months = - 1)
        print(f"Getting TEST data for {date_file.year}-{date_file.month} period")
        test_data = read_file(key = f"datasets/car-prices-{date_file.year}-{date_file.month}.csv")

        return test_data

    else:
        train_data = pd.DataFrame()
        for i in range(periods+1, 1, -1):
            date_file = dt_current + relativedelta(months = - i)
            try:
                data = read_file(key = f"datasets/car-prices-{date_file.year}-{date_file.month}.csv")
                print(f"Getting TRAIN data for {date_file.year}-{date_file.month} period")
            except:
                print(f"Cannot find file car-prices-{date_file.year}-{date_file.month}.csv",
                    "using blank")
                data = None
                
            train_data = pd.concat([train_data, data])
        
        return train_data


def na_filter(data):
    work_data = data.copy()
    non_type = work_data[data['make'].isna() | data['model'].isna() | data['trim'].isna()].index
    work_data.drop(non_type, axis=0, inplace=True)

    y = work_data.pop('sellingprice')

    return work_data, y


class FeaturesModifier:
    def __init__(self, columns):
        self.columns = columns

    def fit(self, work_data, _ = None):
        return self

    def transform(self, work_data, _ = None):

        work_data = pd.DataFrame(work_data, columns = self.columns)
        work_data['make_model_trim'] = work_data['make'] + '_'  + work_data['model'] + '_' + work_data['trim']
        work_data['year'] = work_data['year'].astype('str')
        
        cat_cols = ['year', 'make_model_trim', 'body', 'transmission', 'color', 'interior']
        num_cols = ['condition', 'odometer', 'mmr']

        X = work_data[cat_cols + num_cols].copy()
        X_dict = X.to_dict(orient = 'records')

        return X_dict

    def fit_transform(self, work_data, _ = None):
        return self.transform(work_data)

@task
def prepare_features(work_data, preprocessor = None):

    num_2_impute = ['condition', 'odometer', 'mmr']
    cat_2_impute = ['body', 'transmission']
    constant_2_impute = ['color', 'interior']
    others = ['year', 'make', 'model', 'trim']
    
    if not preprocessor:
        features_filler = ColumnTransformer([
            ('num_imputer', SimpleImputer(missing_values=np.nan, strategy='mean'), num_2_impute),
            ('cat_imputer', SimpleImputer(missing_values=np.nan, strategy='most_frequent'), cat_2_impute),
            ('cat_constant', SimpleImputer(missing_values=np.nan, strategy='most_frequent'), constant_2_impute),
            ('others', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value='-1'), others )
            ]
        )

        fm = FeaturesModifier(columns = num_2_impute + cat_2_impute + constant_2_impute + others)

        dv = DictVectorizer() 

        preprocessor = Pipeline(steps = [
            ('filler', features_filler),
            ('modifier', fm),
            ('dict_vectorizer', dv)

        ])
        
        X = preprocessor.fit_transform(work_data)

    else:
        X = preprocessor.transform(work_data)

    return X, preprocessor

@task
def params_search(train, valid, y_train, y_valid, train_data_file, models):
    
    best_models = []

    for baseline in models:
        
        mlflow.set_experiment(f"{baseline.__name__}-models")
        search_space = models[baseline]

        def objective(params):

            with mlflow.start_run():
                mlflow.set_tag("baseline", f"{baseline.__name__}")
                mlflow.log_param("training-data", train_data_file)
                mlflow.log_param("parameters", params)
                
                print('Serching for the best parameters...')

                training_model = baseline(**params)
                training_model.fit(train, y_train)

                print('Predicting on the valid dataset...')
                prediction_valid = training_model.predict(valid)
                rmse_valid = mean_squared_error(y_valid, prediction_valid, squared = False)

                print('RMSE on valid', 
                    rmse_valid
                    )
                mlflow.log_metric('rmse', rmse_valid)
            

            return {'loss': rmse_valid, 'status': STATUS_OK}
        
        best_result = fmin(fn = objective,
                    space = search_space,
                    algo = tpe.suggest,
                    max_evals = 2, # int(2**(len(models[baseline].items())-2)), #3,
                    trials = Trials(),
                    ) 
        
        print("Best model", baseline(**space_eval(search_space, best_result)))
        best_models.append(baseline(**space_eval(search_space, best_result)))

        mlflow.end_run()
    
    return best_models #ML_model(**space_eval(search_space, best_result))

@task
def train_best_models(train, y_train, X_valid, y_valid, X_test, y_test, preprocessor, models):

    mlflow_client = MlflowClient(tracking_uri = MLFLOW_TRACKING_URI)

    best_pipelines = []

    mlflow.autolog()
    for model in models:
    
        experiment = mlflow.set_experiment(f"{model.__name__}-models")

        best_run = mlflow_client.search_runs(
                experiment_ids = experiment.experiment_id,
                run_view_type=ViewType.ACTIVE_ONLY,
                max_results = 2,
                order_by = ['metrics.rmse ASC']
            )
        
        print(f"Training {model.__name__} with best params")

        mlflow.set_experiment("Auction-car-prices-best-models")
    
        with mlflow.start_run():

            best_params = json.loads(best_run[0].data.params['parameters'].replace("'", "\""))
            staged_model = model(**best_params).fit(train, y_train)
            
            pipeline = Pipeline(
                steps = [
                    ('preprocessor', preprocessor),
                    ('model', staged_model)
                ]
            )
            predict_valid = pipeline.predict(X_valid)
            rmse_valid = mean_squared_error(y_valid, predict_valid, squared = False)

            predict_test = pipeline.predict(X_test)
            rmse_test = mean_squared_error(y_test, predict_test, squared = False)

            mlflow.log_metric("rmse_valid", rmse_valid)
            mlflow.log_metric("rmse_test", rmse_test)
            mlflow.sklearn.log_model(pipeline, artifact_path='full-pipeline')
            
            best_pipelines.append((model.__name__, pipeline))

            print("{:} MODEL was saved with RUN".format(model.__name__))

            mlflow.end_run()

    return best_pipelines

def model_to_registry(MLFLOW_TRACKING_URI, model_name):

    mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
    mlflow_client = MlflowClient(tracking_uri = MLFLOW_TRACKING_URI)
    
    experiment = mlflow.set_experiment('Auction-car-prices-best-models')

    best_model_run = mlflow_client.search_runs(
        experiment_ids=experiment.experiment_id,
        run_view_type=ViewType.ACTIVE_ONLY,
        max_results=1,
        order_by=["metrics.rmse_test ASC"]
            
        )
    RUN_ID = best_model_run[0].info.run_id
    model_uri = "runs:/{:}/full-pipeline".format(RUN_ID)

    # try: 
    #     mlflow_client.get_registered_model("MLOps-project-models")
    # except:
    #     print("no such one, creating")
    #     mlflow_client.create_registered_model("MLOps-project-models")

    # timestamp = datetime.strftime(datetime.today(), "%y-%m-%d_%H:%M")
   
    print("Registering model", model_name)
    mlflow.register_model(
            model_uri=model_uri,
            name = model_name
        )

def model_promotion(current_date, version_to_production, to_stage):

    mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
    mlflow_client = MlflowClient(tracking_uri = MLFLOW_TRACKING_URI)
    model_name = "Auction-car-prices-prediction"

    versions = mlflow_client.get_latest_versions(
        model_name,
        # stages=['Production']
        )

    for version in versions:
        print(f"Model: {version.name:40s} Version: {version.version:3} Stage: {version.current_stage:10s} run_id: {version.run_id:30s}")

    mlflow_client.transition_model_version_stage(
        name = model_name,
        version = version_to_production,
        stage = to_stage,
        archive_existing_versions=False
        )

    mlflow_client.update_model_version(
        name = model_name,
        version = version_to_production,
        description=f'The model was promoted to Production {current_date}'
        )

@flow(task_runner = SequentialTaskRunner())
def main(current_date = "2015-6-20", periods = 5):
 
    train_data = load_data(current_date = current_date, periods = periods)
    X, y = na_filter(train_data)

    test_data = load_data(current_date = current_date)
    X_test, y_test = na_filter(test_data)

    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=42)

    print("Training preprocessor...")
    train, preprocessor = prepare_features(X_train, preprocessor = None )
    valid, _  = prepare_features(X_valid, preprocessor)

    print("Initializing parameters for baseline models")
    models = {
        LinearRegression: {
            "fit_intercept": hp.choice("fit_intercept", ('True', 'False'))
            },
        Ridge: {"alpha": hp.loguniform("alpha", -5, 5),
                "fit_intercept": hp.choice("fit_intercept", ('True', 'False'))
            },
        # RandomForestRegressor: {
        #         'max_depth': scope.int(hp.quniform('max_depth', 1, 20, 1)),
        #         'n_estimators': scope.int(hp.quniform('n_estimators', 10, 50, 1)),
        #         'min_samples_split': scope.int(hp.quniform('min_samples_split', 2, 10, 1)),
        #         'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf', 1, 4, 1)),
        #         'random_state': 42
        #         },
        # XGBRegressor: {
        #         'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
        #         'learning_rate': hp.loguniform('learning_rate', -3, 0),
        #         'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
        #         'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
        #         'max_child_weight': hp.loguniform('max_child_weight', -1, 3),
        #         'num_boost_rounds': 100,
        #         # 'early_stopping_rounds': 20,
        #         'objective': 'reg:squarederror',
        #         'seed': 42,
        #         }
        }

    best_models = params_search(train, valid, y_train, y_valid, current_date, models)

    print("!!!Best params!!!\n", best_models)

    best_pipelines = train_best_models(
        train, y_train,
        X_valid, y_valid,
        X_test, y_test,
        preprocessor,
        models = models
        )

    model_name = "Auction-car-prices-prediction"
    # model_to_registry(MLFLOW_TRACKING_URI, model_name)

    # model_promotion(current_date, version_to_production=3, to_stage = "Production")


/homw/winx/.conda/envs/mlops-project-env/lib/python3.9/site-packages/prefect/tasks.py:166: UserWarning: A task named 'load_data' and defined at '/tmp/ipykernel_7966/80872614.py:57' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/homw/winx/.conda/envs/mlops-project-env/lib/python3.9/site-packages/prefect/tasks.py:166: UserWarning: A task named 'prepare_features' and defined at '/tmp/ipykernel_7966/80872614.py:120' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`
  warnings.warn(
/homw/winx/.conda/envs/mlops-project-env/lib/python3.9/site-packages/prefect/tasks.py:166: UserWarning: A task named 'params_search' and defined at '/tmp/ipykernel_7966/80872614.py:155' conflicts with another task. Consider specifying a unique `name` parameter in the task definition:

 `@task(name='my_unique_name', ...)`

In [13]:
main()

23:33:04.626 | INFO    | prefect.engine - Created flow run 'large-terrier' for flow 'main'
23:33:04.778 | INFO    | Flow run 'large-terrier' - Created task run 'load_data-2ff00c39-0' for task 'load_data'
23:33:04.780 | INFO    | Flow run 'large-terrier' - Executing 'load_data-2ff00c39-0' immediately...


Getting TRAIN data for 2014-12 period
Getting TRAIN data for 2015-1 period
Getting TRAIN data for 2015-2 period
Getting TRAIN data for 2015-3 period
Getting TRAIN data for 2015-4 period


23:33:07.962 | INFO    | Task run 'load_data-2ff00c39-0' - Finished in state Completed()
23:33:08.266 | INFO    | Flow run 'large-terrier' - Created task run 'load_data-2ff00c39-1' for task 'load_data'
23:33:08.267 | INFO    | Flow run 'large-terrier' - Executing 'load_data-2ff00c39-1' immediately...


Getting TEST data for 2015-5 period


23:33:08.590 | INFO    | Task run 'load_data-2ff00c39-1' - Finished in state Completed()
23:33:08.807 | INFO    | Flow run 'large-terrier' - Created task run 'prepare_features-fbf67cd7-0' for task 'prepare_features'
23:33:08.807 | INFO    | Flow run 'large-terrier' - Executing 'prepare_features-fbf67cd7-0' immediately...
2022/08/20 23:33:08 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'd5989ffa6ca34a76915b485b2c701700', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2022/08/20 23:33:08 WARNING mlflow.utils: Truncated the value of the key `steps`. Truncated value: `[('filler', ColumnTransformer(transformers=[('num_imputer', SimpleImputer(),
                                 ['condition', 'odometer', 'mmr']),
                                ('cat_imputer',
                                 SimpleImputer(strategy='most_frequent'),
                                 ['body', 'transm

...

Training LinearRegression with best params
[<Run: data=<RunData: metrics={'mean_squared_error_valid': 1618.6086765949942,
 'rmse': 1618.6086765949942,
 'training_mae': 993.9906453644213,
 'training_mse': 2577775.408513338,
 'training_r2_score': 0.9709853785841451,
 'training_rmse': 1605.5452060011696,
 'training_score': 0.9709853785841451}, params={'copy_X': 'True',
 'fit_intercept': 'False',
 'n_jobs': 'None',
 'normalize': 'deprecated',
 'parameters': "{'fit_intercept': 'False'}",
 'positive': 'False',
 'training-data': '2015-6-20'}, tags={'baseline': 'LinearRegression',
 'estimator_class': 'sklearn.linear_model._base.LinearRegression',
 'estimator_name': 'LinearRegression',
 'mlflow.log-model.history': '[{"run_id": "c76658fea44f45e6af6df19f117e9f51", '
                             '"artifact_path": "model", "utc_time_created": '
                             '"2022-08-20 23:33:18.248176", "flavors": '
                             '{"python_function": {"model_path": "model.pkl", '
   

23:33:35.321 | INFO    | Task run 'train_best_models-1032c79c-0' - Finished in state Completed()


Ridge MODEL was saved with RUN


23:33:37.404 | INFO    | Flow run 'large-terrier' - Finished in state Completed('All states completed.')


[Completed(message=None, type=COMPLETED, result=      year    make                model   trim   body transmission  \
 0     2015     Kia              Sorento     LX    SUV    automatic   
 1     2015     Kia              Sorento     LX    SUV    automatic   
 2     2014     BMW  6 Series Gran Coupe   650i  Sedan    automatic   
 3     2015  Nissan               Altima  2.5 S  Sedan    automatic   
 4     2014     BMW                   M5   Base  Sedan    automatic   
 ...    ...     ...                  ...    ...    ...          ...   
 1445  2004    Jeep              Liberty  Sport    SUV          NaN   
 1446  2001  Nissan               Maxima     SE  Sedan    automatic   
 1447  1997   Lexus               ES 300   Base  Sedan    automatic   
 1448  2015    Ford               Escape     SE    SUV    automatic   
 1449  2014  Toyota              Corolla     LE  Sedan    automatic   
 
                     vin state  condition  odometer   color interior  \
 0     5xyktca69fg566472   

In [6]:
# mlflow.end_run()
# mlflow_client.delete_registered_model("Auction-car-prices-prediction")
# mlflow.delete_experiment(experiment_id=5)


# loaded_model = mlflow.pyfunc.load_model(model_uri=model_uri)
# print(loaded_model)
# loaded_model.predict(X_test)

model_name = "Auction-car-prices-prediction"
stage = 'Production'
production_model = mlflow.pyfunc.load_model(model_uri=f'models:/{model_name}/{stage}')

mlflow.pyfunc.loaded_model:
  artifact_path: full-pipeline
  flavor: mlflow.sklearn
  run_id: 9d39355537eb4504954636659de5b1ec



array([ 2713.04427261, 12171.78593638, 13229.49827207, ...,
       26662.07692966, 23770.28814217,  9173.15714517])